In [ ]:
# from datasets import Dataset, DatasetDict

# # Specify the path to your text file
# text_file_path = "datasets/v1-v2-combine_400mb.txt"

# # Read the text file
# with open(text_file_path, "r", encoding="utf-8") as file:
#     texts = file.readlines()

# # Remove leading and trailing whitespaces from each line
# texts = [text.strip() for text in texts]

# # Create a dictionary with your data
# data_dict = {"text": texts}

# # Create a Hugging Face Dataset
# dataset = Dataset.from_dict(data_dict)

# # Split the dataset into train and test
# # You can customize the split ratio as needed
# train_dataset = dataset.train_test_split(test_size=0.2, seed=42)["train"]
# test_dataset = dataset.train_test_split(test_size=0.2, seed=42)["test"]

# # Create a DatasetDict to store train and test splits
# dataset_dict = DatasetDict({"train": train_dataset, "test": test_dataset})

# # Save the dataset to disk in Hugging Face format
# dataset_dict.save_to_disk("datasets/mizo")

In [ ]:
# from datasets import load_dataset, concatenate_datasets
# from datasets import Dataset
# # Specify the path to the directory containing the dataset files
# dataset_path = "datasets/mizo/train"

# # Load the train and test datasets separately
# train_dataset = Dataset.from_file("datasets/mizo/train/train.arrow")
# test_dataset = Dataset.from_file("datasets/mizo/test/test.arrow")

# # Concatenate train and test datasets into a single variable
# datasets = concatenate_datasets([train_dataset["train"], test_dataset["test"]])

# # Now you can use the 'datasets' variable as a combined dataset
# # For example, printing the first few examples from the combined dataset
# print("Combined Examples:")
# print(datasets[:5])


In [3]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": "datasets/mizo/train.txt", "validation": "datasets/mizo/test.txt"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [5]:
datasets["train"][10]

{'text': 'Hripui leng thawh hnihnain Public health a nuai chhe nasa zual tawh a. Hei vang hian mi tam takin hna an chan a ni” tiin advocate Prashant Bhushan, Centre leh State te migrant worker-te laka inthlahdaha puhtu chuan a sawi a.Bhushan chuan mipui vaibelchhe 24 chu retheiah nawr liamin an awm tawh a. Delhi leh Uttar Pradesh-te ang chuan sorkar school-ahte migrant-te chaw an pe tur a ni tiin a sawi bawk a.“Chaw leh hriselna tha lo avangin mi tam tak an thi a. Nikum aiin dinhmun a chhe zawk a. Rickshaw puller, vendors leh unorganised sector-a awmte chu riltamin an awm a ni” tiin Bhushan chuan judge a hnial a.Bhushan chuan court chu State te chu worker-te tana cash transfer mechanism ruahman tir tura ngenin, “Heng pawisa pekte hi economy a lo let leh tho tur a ni.'}

In [6]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(datasets["train"])

,text
0,"Hemi chungchangah hian inkhapna a awm lo a, an zalennaah kan dah,"""""""" a ti."""
1,The Washington Post chanchinbua Amber-i thuziah chungchanga a lungawi lohna chuan tun hi a thlenpui ta.
2,1116.50Rs.
3,"kal ve tawh a, a thawktute pawh an fakawm ka ti em em a ni Tlawng hnar an enkawl"
4,"Law Minister dan hre mi kan nei hi a thlamuanthlak ka ti a, lawmawm pawh ka ti a. Lawngtlai District Court saw a la phuisui lo em em a."
5,1390.00Rs.
6,
7,"""Robert Romawia Royte chuan, contractor ṭhenkhat, hna thawk muang deuhte chu Covid-19 dinhmun a ziaawm tawh avangin nasa lehzuala ṭan la tura chahin, """"""""Hna thawk chak thei lo an awm chuan an chungah action lak a ni dawn a, hna quality ṭha lo chu ṭhiahtir leh zel an ni ang,"""""""" a ti a; chief minister Zoramthanga’n lung a phùm tawhnaahte hmasawnna hna chak taka thawh vek a nih thu a sawi."""
8,";"""
9,


In [8]:
model_checkpoint = "bert-base-multilingual-uncased"

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [12]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/2789497 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (621 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (573 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (540 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (837 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/697375 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (43609 > 512). Running this sequence through the model will result in indexing errors


In [13]:
tokenized_datasets["train"][1]

{'input_ids': [101, 102], 'token_type_ids': [0, 0], 'attention_mask': [1, 1]}

In [14]:
block_size = tokenizer.model_max_length
# block_size = 128

In [15]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [16]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/2789497 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/697375 [00:00<?, ? examples/s]

In [17]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'##rant worker - te tan nikum tlukin a chhia em tiin rinhlelhna a nei a. [SEP] [CLS] [SEP] [CLS] justice bhushan leh shah - te chuan kum 2020 khan migrant worker - te welfare kaihhnawihah thupek an chhuah nual a ni. [UNK] kuminah chuan nikum ang em emin a rapthlak lem lo [UNK] tiin justice bhushan chuan a sawi a. [UNK] a nihna takah chuan a rapthlak zawk. [SEP] [CLS] [SEP] [CLS] hripui leng thawh hnihnain public health a nuai chhe nasa zual tawh a. hei vang hian mi tam takin hna an chan a ni [UNK] tiin advocate prashant bhushan, centre leh state te migrant worker - te laka inthlahdaha puhtu chuan a sawi a. bhushan chuan mipui vaibelchhe 24 chu retheiah nawr liamin an awm tawh a. delhi leh uttar pradesh - te ang chuan sorkar school - ahte migrant - te chaw an pe tur a ni tiin a sawi bawk a. [UNK] chaw leh hriselna tha lo avangin mi tam tak an thi a. nikum aiin dinhmun a chhe zawk a. rickshaw puller, vendors leh unorganised sector - a awmte chu riltamin an awm a ni [UNK] tiin bhushan chu

In [18]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-mizo-corpus",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

NameError: name 'TrainingArguments' is not defined

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")